In [10]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# Slice 
slice_name = 'Server3'

#[site1,site2,site3] = fablib.get_random_sites(count=3)
site1 = 'UCSD'
site2 = 'DALL'
print(f"Sites: {site1},{site2}")

# Hosts
h1_name = "h1"
h2_name = "h2"

h1_subnet=IPv4Network('192.168.1.0/24')
h1_addr=IPv4Address('192.168.1.2')

h2_subnet=IPv4Network('192.168.2.0/24')
h2_addr=IPv4Address('192.168.2.2')

host_cores = 8
host_ram = 16
host_disk = 500

net_s1_s2_name = 'net_s1_s2'

# All node properties
image = 'default_ubuntu_20'

Sites: UCSD,DALL


In [11]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    

    # Add host node h1
    h1 = slice.add_node(name=h1_name, site=site1, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h1_iface = h1.add_component(model='NIC_Basic', name="h1_nic").get_interfaces()[0]
    
    # Add host node h2
    h2 = slice.add_node(name=h2_name, site=site2, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h2_iface = h2.add_component(model='NIC_Basic', name="h2_nic").get_interfaces()[0]
    #Add swtich networks
    switch_net1 = slice.add_l2network(name=net_s1_s2_name, interfaces=[h1_iface, h2_iface])
    #Submit Slice Request
    slice.submit() 
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    


Retry: 8, Time: 228 sec


ID,f6534eec-1dc4-447b-a92b-b657920ddce4
Name,Server3
Lease Expiration (UTC),2023-08-30 18:14:06 +0000
Lease Start (UTC),2023-08-29 18:14:07 +0000
Project ID,527832fc-c273-4254-b988-16e5c2923bf9
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
05957811-187a-43c0-8ad6-c607cc0c12bb,h1,8,16,500,default_ubuntu_20,qcow2,ucsd-w1.fabric-testbed.net,UCSD,ubuntu,132.249.252.188,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.188,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
1ab103cf-5aad-4add-b4ac-286d6f2e2511,h2,8,16,500,default_ubuntu_20,qcow2,dall-w2.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:feba:af78,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:feba:af78,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
496ec557-4baa-4a5c-9204-0b40675a5b9d,net_s1_s2,L2,L2STS,None,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
h1-h1_nic-p1,p1,h1,net_s1_s2,100,config,,02:74:7A:C2:0A:63,ens7,ens7,None,6
h2-h2_nic-p1,p1,h2,net_s1_s2,100,config,,06:03:79:10:FC:85,ens7,ens7,None,4



Time to print interfaces 231 seconds


In [ ]:
h2.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h1.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')

In [ ]:
h1.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h2.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')

In [ ]:
h3.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h3.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')

In [ ]:
h1.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py --dst_id 2 10.10.2.2 "message100"\'')

In [ ]:
print(h2.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

In [3]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta


slice_name = 'Nagm_P4Test01'
#Set end host to now plus 2 day
end_date = (datetime.now(timezone.utc) + timedelta(days=3)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")